# San Francisco Crime Project

* Author: Kevin Chuang [@k-chuang](https://www.github.com/k-chuang)
* Created on: August 25, 2018
* Description: Data analysis, exploration, visualization, and data mining on crime in SF
* Original dataset: [SF Gov Crime dataset](https://data.sfgov.org/Public-Safety/-Change-Notice-Police-Department-Incidents/tmnf-yvry/about)
* Kaggle dataset: [Kaggle SF Crime](https://www.kaggle.com/c/sf-crime/data)

---------------

# Table of Contents

- Introduction
    - SF Crime Dataset
- Basic Preparation
    - Import libraries
    - Load data
- Data Exploration/Analysis Extension
- Data Preprocessing
    - Data Imputation/Removal
    - Feature Engineering
    - Feature Encoding
- Build Machine Learning Models
    - Train different baseline models
    - Analyze results
- Model Selection
- Hyperparameter tuning
- Train Model with optimal hyperparameters
- Feature Selection
    - Feature Importance
    - Feature Removal
- Train Final Model
- Model Evaluation
- Summary
- Kaggle Submission
- Conclusion

# Introduction


## SF Crime Dataset

This dataset contains incidents derived from SFPD Crime Incident Reporting system. The data ranges from 1/1/2003 to 5/13/2015. The training set and test set rotate every week, meaning week 1,3,5,7... belong to test set, week 2,4,6,8 belong to training set. The goal is to try to predict the category of crime that occurred in the city of San Francisco. 

### Data Fields
- **Dates** - timestamp of the crime incident
- **Category** - category of the crime incident (only in train.csv). This is the target variable you are going to predict.
- **Descript** - detailed description of the crime incident (only in train.csv)
- **DayOfWeek** - the day of the week
- **PdDistrict** - name of the Police Department District
- **Resolution** - how the crime incident was resolved (only in train.csv)
- **Address** - the approximate street address of the crime incident 
- **X** - Longitude
- **Y** - Latitude


In this juypter notebook, I will go through the whole process, end-to-end, of creating a machine learning model on the open source San Francisco Crime dataset. This includes data exploration & analysis, data preprocessing (huge part of this project and includes feature engineering), trying out different ML algorithms and determining the optimal ML model, tuning the hyperparameters of that model, and finally, evaluating the chosen model in terms of multiclass log loss. 

Since this is an old Kaggle competition, I will refrain from looking online for resources or old Kaggle kernels. The plan is to get better at coding an end to end data science project and to familiarize myself with the Python data science libraries. Also, I hope to learn some interesting things and discover some cool patterns or ideas using this dataset. Well, here goes nothing!



## Import libraries

In [ ]:
__author__ = 'Kevin Chuang (https://www.github.com/k-chuang)' 

# linear algebra
import numpy as np 

# data processing
import pandas as pd 

# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

# Algorithms
from sklearn import linear_model
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import xgboost as xgb

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

# Metrics 
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score

# Model Selection & Hyperparameter tuning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold
from skopt import BayesSearchCV
from skopt.space  import Real, Categorical, Integer


# Clustering
from sklearn.cluster import KMeans

# Mathematical Functions
import math
import re

## Load data

In [ ]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [ ]:
train_df.info()

# Data Exploration & Analysis Extension

- Complete data exploration & visualizations are located in jupyter notebook: [kaggle-sf-crime-data-exploration.ipynb](kaggle-sf-crime-data-exploration.ipynb)
- This dataset suffers from **imbalanced classes** (**TREA** has 6 occurrences while **LARCENY/THEFT** has 1,749,000 occurrences)
    - There are a couple ways to deal with imbalanced classes, such as:
        - Changing performance metric (Do not use accuracy, use a confusion matrix, precision, recall, F1 score, ROC curves)
        - Resample dataset (Oversample under-represented classes, and undersample over-represented classes)
        - Try different ML algorithms that can handle imbalanced classes
            - Decision Trees (Random Forests/XGBoost) often perform well on imbalanced classes (due to splitting rules)

In [ ]:
train_df.head(8)

In [ ]:
train_df.columns.values

In [ ]:
# set show nulls to True
train_df.info(verbose=True, null_counts=True)

------------
### Things we learned thus far:

- 878,049 instances in training set (or recorded crime instances in SF)
- 9 columns (8 potential features + 1 label (Category))
- Data types:
    - 2 columns with float values
    - 7 objects
- There are no null (NaN) values (Yay!)

In [ ]:
## Count number of observations for each crime 
train_df['Category'].value_counts()

In [ ]:
## Count number of observations of crime for each PD District
train_df['PdDistrict'].value_counts()

In [ ]:
## Count number of observations for each day of week
train_df['DayOfWeek'].value_counts()

In [ ]:
## Count number of observations for Resolution feature
train_df['Resolution'].value_counts()

In [ ]:
train_df[['X','Y']].describe()

**There seems to be an invalid coordinates (max) 90 (latitude) or -120.5 (longitude) does not seem to be a valid coordinate in San Francisco. We must fix these values for this feature.**

# Data Preprocessing

- Data cleaning
    - imputation or removal of outlier values
- Feature Engineering (Feature Creation)
- Feature Encoding
    - **Integer encode** or **label encode** ordinal categorical features that maintain order (Year, Business Quarter, Block/Street Number)
    - Usually: 
        - **One hot encode** nominal categorical features (DayOfWeek, PdDistrict, StreetType, Category)
            - mainly for logistic regression
        - However, Random Forests & Boosting algorithms can handle nominal categorical features directly, so we just **integer encode** these features.

## Data Cleaning

- Data removal
- Data imputation

In [ ]:
train_df[train_df['Y'] == train_df['Y'].max()]

I notice that there are 108 rows with incorrect coordinates, and they seem to be the exact same two coordinates (90, -120.5). There are many ways to handle this. We need to do data imputation, which can be done several ways. For now, I will randomly sample from a normal distribution with the range of a standard deviation from the mean. However, I could use a linear regression model to predict the latitude and longitude values (based on other variables such as PD district?) and use that to impute the bad / inconsistent data points.

Another method is to completely remove this data. Since I already have a lot of data, and I do not want this incorrect data to affect my results, I could remove them. However, I will stick with data imputation.

In [ ]:
train_df['Y'].replace(to_replace= train_df['Y'].max() ,value=np.nan, inplace=True)
train_df['X'].replace(to_replace= train_df['X'].max() ,value=np.nan, inplace=True)
test_df['Y'].replace(to_replace= test_df['Y'].max() ,value=np.nan, inplace=True)
test_df['X'].replace(to_replace= test_df['X'].max() ,value=np.nan, inplace=True)

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

In [ ]:
data = [train_df, test_df]

for dataset in data:
    mean_X = dataset["X"].mean()
    std_X = dataset["X"].std()
    mean_Y = dataset["Y"].mean()
    std_Y = dataset["Y"].std()
    max_X = mean_X + std_X
    min_X = mean_X - std_X
    max_Y = mean_Y + std_Y
    min_Y = mean_Y - std_Y

    # Both X and Y will have the same null so just use Y
    is_null = dataset['Y'].isnull().sum()
    random_X = (max_X - min_X) * np.random.randn(is_null) + min_X
    random_Y = (max_Y - min_Y) * np.random.randn(is_null) + min_Y

    X_slice = dataset['X'].copy()
    Y_slice = dataset['Y'].copy()
    X_slice[np.isnan(X_slice)] = random_X
    Y_slice[np.isnan(Y_slice)] = random_Y
    dataset['X'] = X_slice
    dataset['Y'] = Y_slice


In [ ]:
train_df[['X', 'Y']].describe()

In [ ]:
len(train_df)

In [ ]:
test_df[['X', 'Y']].describe()

In [ ]:
len(test_df)

# Feature Engineering

- Let's create some new features from the data that exists in the current feature space
- There are a couple categories of features:
    - Temporal features
    - Spatial features

## Temporal Features
We want to have a column for Time, so we must parse through the 'Dates' feature to create the 'Time' feature


In [ ]:
# Transform the Date into a python datetime object.
train_df["Dates"] = pd.to_datetime(train_df["Dates"], format="%Y-%m-%d %H:%M:%S")
test_df["Dates"] = pd.to_datetime(test_df["Dates"], format="%Y-%m-%d %H:%M:%S")

In [ ]:
# Minute
train_df["Minute"] = train_df["Dates"].map(lambda x: x.minute)
test_df["Minute"] = test_df["Dates"].map(lambda x: x.minute)

In [ ]:
# Hour
train_df["Hour"] = train_df["Dates"].map(lambda x: x.hour)
test_df["Hour"] = test_df["Dates"].map(lambda x: x.hour)

In [ ]:
# Day
train_df["Day"] = train_df["Dates"].map(lambda x: x.day)
test_df["Day"] = test_df["Dates"].map(lambda x: x.day)

In [ ]:
# Month
train_df["Month"] = train_df["Dates"].map(lambda x: x.month)
test_df["Month"] = test_df["Dates"].map(lambda x: x.month)

In [ ]:
# Year
train_df["Year"] = train_df["Dates"].map(lambda x: x.year)
test_df["Year"] = test_df["Dates"].map(lambda x: x.year)

In [ ]:
# Hour Zone 0 - Pass midnight, 1 - morning, 2 - afternoon, 3 - dinner / sun set, 4 - night
def get_hour_zone(hour):
    if hour >= 2 and hour < 8: 
        return 0
    elif hour >= 8 and hour < 12: 
        return 1
    elif hour >= 12 and hour < 18: 
        return 2
    elif hour >= 18 and hour < 22: 
        return 3
    elif hour < 2 or hour >= 22: 
        return 4
    
train_df["Hour_Zone"] = train_df["Hour"].map(get_hour_zone)
test_df["Hour_Zone"] = test_df["Hour"].map(get_hour_zone)

In [ ]:
# Add Week of Year
train_df["WeekOfYear"] = train_df["Dates"].map(lambda x: int(x.weekofyear / 2) - 1)
test_df["WeekOfYear"] = test_df["Dates"].map(lambda x: int(x.weekofyear / 2))

print(sorted(train_df['WeekOfYear'].unique()))
print(sorted(test_df['WeekOfYear'].unique()))

In [ ]:
train_df.head()

### Holiday Feature

- Certain crimes may be more apparent on holidays

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

# Training set
cal = calendar()
holidays = cal.holidays(start=train_df['Dates'].min(), end=train_df['Dates'].max())
train_df['Holiday'] = train_df['Dates'].dt.date.astype('datetime64').isin(holidays)

In [ ]:
# Test set
cal = calendar()
holidays = cal.holidays(start=test_df['Dates'].min(), end=test_df['Dates'].max())
test_df['Holiday'] = test_df['Dates'].dt.date.astype('datetime64').isin(holidays)

In [ ]:
len(train_df[train_df['Holiday'] == True])

In [ ]:
len(test_df[test_df['Holiday'] == True])

### Business Hours Feature

- There should be an effect of business hours on the type of crime committed
- Let's create a binary feature where:
    - 1 is typical business hours [8:00AM - 6:00PM]
    - 0 is not business hours [6:01PM - 7:59 AM]

In [ ]:
from datetime import datetime, time

def time_in_range(start, end, x):
    """Return true if x is in the inclusive range [start, end]"""
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end

def map_business_hours(date):
    
    # Convert military time to AM & PM
    time_parsed = date.time()
    business_start = time(8, 0, 0)
    business_end = time(18, 0, 0)
    
    if time_in_range(business_start, business_end, time_parsed):
        return 1
    else:
        return 0
    
train_df['BusinessHour'] = train_df['Dates'].map(map_business_hours).astype('uint8')
test_df['BusinessHour'] = test_df['Dates'].map(map_business_hours).astype('uint8')

In [ ]:
train_df['BusinessHour'].value_counts()

In [ ]:
train_df.head(10)

### Season

The season feature may affect what type of crimes are commited. 
- 1 = Winter, 2 = Spring, 3 = Summer, 4 = Fall

In [ ]:
train_df['Season']=(train_df['Month']%12 + 3)//3
test_df['Season']=(test_df['Month']%12 + 3)//3

In [ ]:
train_df.head()

### Weekend

- Weekends may have effect on what types of crimes are commmited
- Weekday = 0, Weekend =1

In [ ]:
# Weekend Feature

# Weekday = 0, Weekend = 1
days = {'Monday':0 ,'Tuesday':0 ,'Wednesday':0 ,'Thursday':0 ,'Friday':0, 'Saturday':1 ,'Sunday':1}

train_df['Weekend'] = train_df['DayOfWeek'].replace(days).astype('uint8')
test_df['Weekend'] = test_df['DayOfWeek'].replace(days).astype('uint8')

## Spatial Features

### Street Type

The street type can have an effect on what type of crime is committed, so we want to extract the street type from the 'Address' feature.

We have avenues, streets, ways, boulevards, highways, courts, walks, plazas, and differet number of intersections of roads/streets (Addresses with /).

In [ ]:
train_df['Address'].value_counts().index

In [ ]:
def find_streets(address):
    street_types = ['AV', 'ST', 'CT', 'PZ', 'LN', 'DR', 'PL', 'HY', 
                    'FY', 'WY', 'TR', 'RD', 'BL', 'WAY', 'CR', 'AL', 'I-80',  
                    'RW', 'WK','EL CAMINO DEL MAR']
    street_pattern = '|'.join(street_types)
    streets = re.findall(street_pattern, address)
    if len(streets) == 0:
        # Debug
#         print(address)
        return 'OTHER'
    elif len(streets) == 1:
        return streets[0]
    else:
#         print(address)
        return 'INT'

train_df['StreetType'] = train_df['Address'].map(find_streets)
test_df['StreetType'] = test_df['Address'].map(find_streets)


In [ ]:
train_df['StreetType'].value_counts()

In [ ]:
# Check for null values
train_df['StreetType'].isnull().sum()

In [ ]:
train_df.head(8)

## Block Number Feature

- Let's explore the block number from address
- Block number has ordinal data type (order matters), and has spatial significance
- It seems all the block numbers are in intervals of 100
- How to categorize
    - Addresses that do not have a block number will be categorized as 0
    - Addresses with block number will be divided by 100, and added by 1 for mapping (0 is saved for addresses with no block number)
- 85 unique block numbers (including 1 where there is no block number)

In [ ]:
def find_block_number(address):
    block_num_pattern = '[0-9]+\s[Block]'
    block_num = re.search(block_num_pattern, address)
    if block_num:
#         print(address)
        num_pattern = '[0-9]+'
        block_no_pos = re.search(num_pattern, address)
        # Get integer of found regular expression
        block_no = int(block_no_pos.group())
        # Convert block number by dividing by 100 and adding 1 (0 = addresses with no block)
        block_map = (block_no // 100) + 1
#         print(block_map)
        return block_map
    else:
#         print(address)
        # 
        return 0


train_df['BlockNo'] = train_df['Address'].map(find_block_number)
test_df['BlockNo'] = test_df['Address'].map(find_block_number)

In [ ]:
train_df['BlockNo'].value_counts()

## X, Y Coordinates

- Normalize and scale the X and Y coordinates
- I use **K-Means clustering** to create a new feature for the longitude and latitude by grouping clusters of points based on Euclidean distances.
- X = longitude, Y = latitude
- I also extract more spatial features from the X, Y coordinates by transforming them from the cartesian space to the polar space ([Reference](https://www.kaggle.com/c/sf-crime/discussion/18853))
    1. three variants of rotated Cartesian coordinates (rotated by 30, 45, 60 degree each) 
    2. Polar coordinates (i.e. the 'r' and the angle 'theta')
    3. The approach makes some intuitive sense i.e. that having such features should help in extracting some more spatial information (than relying on the current x-y alone)

In [ ]:
# Normalize X and Y
print('There are %d unique longitude values, %d unique latitude values' % (train_df['X'].nunique(), 
                                                                           train_df['Y'].nunique()))

xy_scaler = StandardScaler().fit(train_df[['X', 'Y']])
train_df[['X', 'Y']] = xy_scaler.transform(train_df[['X', 'Y']])
test_df[['X', 'Y']] = xy_scaler.transform(test_df[['X', 'Y']])

In [ ]:
# X-Y plane rotation and space transformation to extract more spatial information
# 2-dimensional rotation based on below functions:
# rotated x = xcos - ysin
# rotated y = xsin + ycos
# Conver from cartesian space -> polar space

cos_30 = math.cos(math.radians(30))
sin_30 = math.sin(math.radians(30))
cos_45 = math.cos(math.radians(45))
sin_45 = math.sin(math.radians(45))
cos_60 = math.cos(math.radians(60))
sin_60 = math.sin(math.radians(60))


train_df["Rot30_X"] = train_df['X'] * cos_30 - train_df['Y'] * sin_30 
train_df["Rot30_Y"] = train_df['X'] * sin_30 + train_df['Y'] * cos_30
train_df["Rot45_X"] = train_df['X'] * cos_45 - train_df['Y'] * sin_45  
train_df["Rot45_Y"] = train_df['X'] * sin_45 + train_df['Y'] * cos_45
train_df["Rot60_X"] = train_df['X'] * cos_60 - train_df['Y'] * sin_60  
train_df["Rot60_Y"] = train_df['X'] * sin_60 + train_df['Y'] * cos_60
train_df["Radius"] = np.sqrt(train_df['X'] ** 2 + train_df['Y'] ** 2)
train_df["Angle"] = np.arctan2(train_df['X'], train_df['Y'])

test_df["Rot30_X"] = test_df['X'] * cos_30 - test_df['Y'] * sin_30  
test_df["Rot30_Y"] = test_df['X'] * sin_30 + test_df['Y'] * cos_30
test_df["Rot45_X"] = test_df['X'] * cos_45 - test_df['Y'] * sin_45  
test_df["Rot45_Y"] = test_df['X'] * sin_45 + test_df['Y'] * cos_45
test_df["Rot60_X"] = test_df['X'] * cos_60 - test_df['Y'] * sin_60  
test_df["Rot60_Y"] = test_df['X'] * sin_60 + test_df['Y'] * cos_60
test_df["Radius"] = np.sqrt(test_df['X'] ** 2 + test_df['Y'] ** 2)
test_df["Angle"] = np.arctan2(test_df['X'], test_df['Y'])

In [ ]:
# View the description of the numerical features again to ensure everything is right
train_df.describe()

In [ ]:
# run KMeans separately on both the training set and test set
data = [train_df, test_df]
num_clusters = 40
for dataset in data:
    coordinates = dataset.loc[:,['Y','X']]
    kmeans = KMeans(n_clusters=num_clusters, random_state=1).fit(coordinates)
    id_labels=kmeans.labels_
#     print(kmeans.cluster_centers_)
    dataset['Cluster'] = id_labels

In [ ]:
train_df.head()

## Drop Features

- We have already extracted all the necessary features from the `Address` attribute, so drop
- We don't need `Resolution` or `Descript` features since it is not included in the training data

In [ ]:
# Drop Address feature from both train and test set
train_df.drop(['Address'], axis=1, inplace=True)
test_df.drop(['Address'], axis=1, inplace=True)

In [ ]:
# We don't need Dates column anymore
train_df.drop(['Dates'], axis=1, inplace=True)
test_df.drop(['Dates'], axis=1, inplace=True)

In [ ]:
# Drop Resolution column since test set does not have this column
train_df.drop(['Resolution'], axis=1, inplace=True)

In [ ]:
# Drop Descript column since test set does not have this column
train_df.drop(['Descript'], axis=1, inplace=True)

In [ ]:
# Let's quickly view the data
train_df.head()

# Feature Encoding 

- Convert categorical data to numeric data

### Pd Districts

- convert Pd District categorical feature to numeric

In [ ]:
pd_districts = {'SOUTHERN':0, 'MISSION':1, 'NORTHERN':2, 'CENTRAL':3, 'BAYVIEW':4, 'INGLESIDE':5, 
                'TENDERLOIN':6, 'TARAVAL':7, 'PARK':8, 'RICHMOND':9}

train_df['PdDistrict'].replace(pd_districts, inplace=True)
test_df['PdDistrict'].replace(pd_districts, inplace=True)

In [ ]:
train_df.head()

In [ ]:
train_df.info()

### Year

- Year is an **ordinal** variable, so let's keep that ordering and mapping
- convert Year categorical feature to numeric

In [ ]:
data = [train_df, test_df]

for dataset in data:
    year_le = LabelEncoder()
    year_le.fit(dataset['Year'].unique())
    print(list(year_le.classes_))

    dataset['Year']=year_le.transform(dataset['Year']) 

In [ ]:
train_df['Year'].unique()

In [ ]:
# So we know the mapping (important)
dict(zip(year_le.classes_, year_le.transform(year_le.classes_)))

In [ ]:
train_df.head()

In [ ]:
train_df.info()

### DayOfWeek

- we are going to use sklearn's LabelEncoder to encode the categorical data to numeric
- Day of week is considered a categorical and nominal variable

In [ ]:
data = [train_df, test_df]

for dataset in data:
    dow_le = LabelEncoder()
    dow_le.fit(dataset['DayOfWeek'].unique())
    print(list(dow_le.classes_))
    dataset['DayOfWeek']=dow_le.transform(dataset['DayOfWeek'])

In [ ]:
train_df['DayOfWeek'].unique()

In [ ]:
# So we know the mapping (important)
dict(zip(dow_le.classes_, dow_le.transform(dow_le.classes_)))

In [ ]:
train_df.head()

In [ ]:
train_df.info()

### Street Type

- we are going to use sklearn's LabelEncoder to encode the categorical data to numeric

In [ ]:
data = [train_df, test_df]

for dataset in data:
    st_le = LabelEncoder()
    st_le.fit(dataset['StreetType'].unique())
    print(list(st_le.classes_))
    dataset['StreetType']=st_le.transform(dataset['StreetType'])

In [ ]:
train_df['StreetType'].unique()

In [ ]:
train_df.head()

In [ ]:
train_df.info()

### Holiday

- Encode the binary feature

In [ ]:
# Encode to 0 and 1

train_df['Holiday'].replace(False, 0, inplace=True)
train_df['Holiday'].replace(True, 1, inplace=True)
test_df['Holiday'].replace(False, 0, inplace=True)
test_df['Holiday'].replace(True, 1, inplace=True)

train_df['Holiday'] = train_df['Holiday'].astype('uint8')
train_df['Holiday'] = train_df['Holiday'].astype('uint8')

In [ ]:
train_df[train_df['Holiday'] == 1].head()

In [ ]:
test_df[test_df['Holiday'] == 1].head()

### Category

- we are going to use sklearn's LabelEncoder to encode the categorical data to numeric

In [ ]:
data = [train_df]

for dataset in data:
    cat_le = LabelEncoder()
    cat_le.fit(dataset['Category'].unique())
    print(list(cat_le.classes_))
    dataset['Category']=cat_le.transform(dataset['Category'])

In [ ]:
len(train_df['Category'].unique())

In [ ]:
# So we know the mapping (important)
dict(zip(cat_le.classes_, cat_le.transform(cat_le.classes_)))

In [ ]:
train_df.head()

In [ ]:
train_df.info()

## View Information about Data

- One last check before training

In [ ]:
train_df.info()

In [ ]:
# # Convert all to 32 bit integers so less memory and will train faster (no loss in data since our integers dont reach)
columns_to_convert = ['DayOfWeek', 'PdDistrict', 'Minute', 'Hour', 'Day', 'Month', 'Year', 
                      'Hour_Zone', 'WeekOfYear', 'Season', 'StreetType', 'BlockNo', 'Cluster']
train_df[columns_to_convert] = train_df[columns_to_convert].astype('int16')
test_df[columns_to_convert] = test_df[columns_to_convert].astype('int16')

train_df.info()

# Building Machine Learning Models

- Baseline Models
    - Let's train a couple models on a stratified sample of the training data
    - Evaluate on a hold out set to get baseline results for each model to determine what model to use
    - Models:
        - Stochastic Gradient Descent (with elastic net regularization)
        - Gaussian Naive Bayes
        - K Nearest Neighbors
        - Logistic Regression (with L1 regularization)
        - Random Forest
        - XGBoost
    - Almost all the default scikit-learn ML algorithm hyperparameters exhibit bad performance
        - Researched online & read literature to determine some more ideal default hyperparameters
            - [Reference](https://arxiv.org/abs/1708.05070)
- Couple things to note:
    - **Decision tree models** including Ensemble methods (Random Forest & XGBoost) can handle categorical variables without one-hot encoding them. 
    - **Linear models** (SGD & Logistic Regression) cannot handle categorical features & need features to be OHE before training
    - Always OneHotEncode before you split data up to training/dev/test so that all features & classes will be represented

In [ ]:
# Set training data (drop labels) and training labels
X_train = train_df.drop("Category", axis=1).copy()
Y_train = train_df["Category"].copy()

# Set testing data (drop Id)
X_test = test_df.drop("Id", axis=1).copy()

In [ ]:
# Use these for ML algorithms that can handle categorical data without OHE
mini_train_data, mini_dev_data, mini_train_labels, mini_dev_labels = train_test_split(X_train, 
                                                                                      Y_train,
                                                                                      stratify=Y_train,
                                                                                      test_size=0.5,
                                                                                      random_state=1)

# Hyperparameter Tuning

- Hyperparameter tuning involves defining an objective function (log loss), and using cross-validation to measure the hyperparameter quality. 
    - We want the hyperparameters that give the highest generalization performance.
- Three approaches: Grid Search (`GridSearchCV`), Random Search (`RandomSearchCV`), and Bayes Optimization (`BayesSearchCV`)
- Realized `GridSearchCV` took way too long and was impractical, and `RandomSearchCV` was too random.
    - Grid and random search are completely uninformed by past evaluations, and as a result, often spend a significant amount of time evaluating “bad” hyperparameters.
- Then, I did more research on more efficient & smarter hyperparameter tuning techniques and found Bayeisan Optimization (`BayesSearchCV`)
- **Bayesian Optimization Overview**
    - Build a probabilistic model of the objective function & use it to select promising hyperparameters to evaluate in the true objective function
        - The model used for approximating the objective function is called *surrogate model*. 
            - E.g. Gaussian Processes 
    - Keeps track of past evaluation results, which is used to form a probabilistic model mapping hyperparameters to a probability of a score on the objective function
    - Instead of optimizing an expensive objective function, we optimize on a cheap proxy function instead.
        - *Acquisition function* that directs sampling to areas where an improvement over the current best observation is likely.
            - E.g. maximum probability of improvement (MPI), expected improvement (EI) and upper confidence bound (UCB)
- **K-Folds Cross Validation**
    - Use cross validation to measure the true generalization performance of a model 
    - This is integrated with the hyperparameter tuning techniques (`GridSearchCV`, `RandomSearchCV`, `BayesSearchCV`)

## XGBoost (Boosting)

- Basic Overview:
    - Another ensemble method that uses Boosting instead of Bagging (Random Forests)
    - In **Boosting**, the trees are built sequentially such that each subsequent tree aims to reduce the errors of the previous tree.
    - Each tree learns from its predecessors and updates the residual errors. 
    - Each base learner is weak (high bias) and contributes some vital information for prediction, enabling the boosting technique to produce a strong learner by effectively combining these weak learners.
    - The final strong learner brings down both the **bias** and the **variance**.
    - In contrast to bagging techniques like Random Forest, in which trees are grown to their maximum extent, boosting makes use of trees with fewer splits
        -  Such small trees, which are not very deep, are **highly interpretable**. 
- Basic Steps:
    1. Initial model `F0` to predict target variable `y`. Used to also calculate residual (`y - F0`)
    2. A new model `h1` is used to fit to the residuals from the previous step
    3. Now, `F0` and `h1` are combined to give `F1`, which is the boosted version of `F0`. 
        - The MSE or whatever cost function you use (Log loss, MAE) of `F1` will be lower than `F0`.
    4. Iterate the above steps to create new models based off the previous models.
    
### Prevent Overfitting:
- Large number of trees will cause overfitting (unlike Random Forests)


# Train model with optimal hyperparameters & all features

- Initially, I started with a Random Forest, but decided to use XGBoost
- We first train the model (with all the features) using the optimal hyperparameters that were found through `BayesSearchCV`
- Then, I use the model to predict the probabilities of test set with all the features
    - I'll save these predictions later to compare them with another model I will train with certain features removed

In [ ]:
# It seems running time scales quadratically with the number of classes
xgb = XGBClassifier(
    n_estimators=86, 
    objective="multi:softprob", 
    learning_rate=0.1858621466840661,
    colsample_bylevel=1.0,
    colsample_bytree=1.0,
    gamma=0.49999999999999994,
    max_delta_step=0,
    max_depth=50,
    min_child_weight=5,
    reg_alpha=1.0,
    reg_lambda=60.121460571845695,
    scale_pos_weight=1e-06,
    subsample=1.0,
    random_state=1, 
    tree_method="gpu_hist",
    gpu_id=0,
    n_jobs=4,
    verbosity=0)


xgb.fit(X_train, Y_train)

Y_test_pred = xgb.predict_proba(X_test)

In [ ]:
import sklearn
print(sklearn.metrics.SCORERS.keys())

# Feature Importance

- Measured by mean decrease in Gini information
- This is a form of feature selection that ensemble methods (Random Forest, XGBoost) can use to prevent overfitting
    - I drop the features that seem unimportant & with less than a 1% contribution

In [ ]:
importances = pd.DataFrame({'feature': X_train.columns,
                            'importance': np.round(xgb.feature_importances_, 5)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')

In [ ]:
importances

# Model Evaluation

- Evaluate final model based on K-Fold cross validation
- Average all K iterations to give the true estimate of the final model's performance

In [ ]:
scores = cross_val_score(xgb, X_train, Y_train, 
                         cv=StratifiedKFold(n_splits=5), 
                         scoring = "neg_log_loss", n_jobs=6)

In [ ]:
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

# Kaggle Submission

- Reformat and turn in predictions and results from our model

In [ ]:
Y_test_pred.shape

In [ ]:
sample_submission = pd.read_csv('data/sampleSubmission.csv')

In [ ]:
sample_submission.shape

In [ ]:
sample_submission.iloc[:, 1:] = pd.DataFrame(Y_test_pred, columns=sample_submission.columns[1:])

In [ ]:
sample_submission.head(10)

In [ ]:
sample_submission.to_csv('data/submission_baseline.csv', index=False)

# Summary

- After lots of tuning, I finally achieved a kaggle evaluation score (multiclass log loss) of **2.25674**, which would ideally rank at **#136** (out of 2,335 teams) or at the **top 6%** or **94th percentile** on the public leaderboard
    - Since this is an old kaggle competition, this would most likely be a lower rank, but I still felt proud to achieve this score
    - It is possible that I could run more experiments and tune the hyperparameters to achieve an even better score & ranking 
    - This was more of a learning experience for me & to get my feet wet with Data Science projects & Kaggle competitions
    - In an effort to learn, I refrained from looking up old Kaggle kernels & other resources that completed this specific Kaggle competition.
    - I coded most of this myself to learn the data science libraries, but did use resources such as other Kaggle competition kernels and research papers to get a better idea of how to think about the data. Google is awesome.
- Below, I show images of my two highest scoring submissions on Kaggle

# Conclusion

This project has taught me a lot about data science and has given me hands-on experience with working with data and completing an end-to-end data science project. I've had a lot of fun visualizing, analyzing, and experimenting with the data to gain more insight. This is just the beginning of my journey into data science, and I am very excited to see what the future holds in terms of new and interesting data science problems and datasets.

- **What I learned**:
    - There are more efficient ways to label or integer encode features
        - Will use sklearn's LabelEncoder, OneHotEncoder, & MultiLabelBinarizer next time
    - Instead of just blindly training models, research more about ways to optimize the hyperparameters efficiently
        - Spent too many AWS EC2 hours with `GridSearchCV`, when I should have used *Bayesian Optimization* for efficient hyperparameter tuning
        - Do more research on the domain of the problem, certain core ML algorithms, and data processing techniques
- **What's next?**
    - AutoML with `tpot` or `auto-sklearn`
        - automate the hyperparameter tuning and model selection with AutoML packages
    - Problem Redirection (Classification ---> Regression)
        - Instead of predicting category of crime, predict X & Y coordinates (longitude & latitude) continuous values given same spatial and temporal features as well as category of crime
        - **Use case:** Dynamically concentrate police on certain serious categories of crime to prevent crimes from happening beforehand
    - Rewrite all code in the jupyter notebook to .py files
        - Modularize each of the steps with functions and/or classes
        - Useful because I can run the .py file on AWS EC2 without having to host it on jupyter notebook locally
            - Meaning I can peacefully shut down my laptop and let script run in the cloud overnight


In [ ]:
""" 12/9 """

cross_valid_result = []
valid_data_size = int(np.ceil(X_train.values.shape[0]/5))

for i in range(5):
    s, e = valid_data_size*i, valid_data_size*(i+1)
    xgb = XGBClassifier(
        early_stopping_rounds=10,
        n_estimators=300, 
        objective="multi:softprob", 
        learning_rate=0.1858621466840661,
        colsample_bylevel=1.0,
        colsample_bytree=1.0,
        gamma=0.49999999999999994,
        max_delta_step=0,
        max_depth=50,
        min_child_weight=5,
        reg_alpha=1.0,
        reg_lambda=60.121460571845695,
        scale_pos_weight=1e-06,
        subsample=1.0,
        random_state=1, 
        n_jobs=4,
        tree_method='gpu_hist', 
        gpu_id = 0,
        silent=False)
    X = pd.concat([X_train[:s],X_train[e:]])
    Y = pd.concat([Y_train[:s],Y_train[e:]])

    xgb.fit(X, Y, eval_set=[(X_train[s:e], Y_train[s:e])])

    cross_valid_result.append(xgb.predict_proba(X_test))